## Annotated in Pydantic


#### 1. "Old Way" (Repetitive & Messy)


In [ ]:
from pydantic import BaseModel, field_validator

# ❌ THE CLUTTERED WAY
# We have to write the same validator twice!


class User(BaseModel):
    username: str
    age: int

    @field_validator("age")
    @classmethod
    def check_positive(cls, v: int) -> int:
        if v < 0:
            raise ValueError("Must be positive")
        return v


class Product(BaseModel):
    name: str
    price: int

    # REPEATED LOGIC 😫
    @field_validator("price")
    @classmethod
    def check_positive(cls, v: int) -> int:
        if v < 0:
            raise ValueError("Must be positive")
        return v

---

#### 2. "Annotated" Way (Write Once, Use Everywhere)


In [1]:
from typing import Annotated
from pydantic import BaseModel, AfterValidator


# 1. Define the validation logic as a standard function
def check_positive(v: int) -> int:
    if v < 0:
        raise ValueError("Must be positive")
    return v


# 2. Create a Reusable Type using Annotated
# Syntax: Annotated[TYPE, VALIDATOR]
PositiveInt = Annotated[int, AfterValidator(check_positive)]


# ✅ THE CLEAN WAY
class User(BaseModel):
    username: str
    # Logic is built into the type!
    age: PositiveInt


class Product(BaseModel):
    name: str
    # Reusing the exact same type
    price: PositiveInt

In [2]:
from pydantic import ValidationError
from pprint import pprint

try:
    p = Product(name="Bad Item", price=-50)
except ValidationError as e:
    pprint(e.json())

('[{"type":"value_error","loc":["price"],"msg":"Value error, Must be '
 'positive","input":-50,"ctx":{"error":"Must be '
 'positive"},"url":"https://errors.pydantic.dev/2.12/v/value_error"}]')


In [3]:
p = Product(name="Good Item", price=50)
print(p)

name='Good Item' price=50


---

#### 3. Combining Multiple Validators (Stacking)


In [4]:
from typing import Annotated
from pydantic import BaseModel, AfterValidator


def to_lower(v: str) -> str:
    return v.lower()


def remove_spaces(v: str) -> str:
    if " " in v:
        raise ValueError("Must not contain spaces")
    return v


# A "Slug" is a string that is lowercase AND has no spaces
# Pydantic runs these left-to-right
Slug = Annotated[str, AfterValidator(to_lower), AfterValidator(remove_spaces)]


class BlogPost(BaseModel):
    title: str
    url_slug: Slug

In [5]:
# It will automatically lower-case the input
post = BlogPost(title="My Day", url_slug="MY-DAY-TRIP")
print(f"Clean Slug: {post.url_slug}")

Clean Slug: my-day-trip


In [6]:
from pydantic import ValidationError
from pprint import pprint

# It will still catch errors
try:
    BlogPost(title="Error", url_slug="my day trip")
except ValidationError as e:
    pprint(e.json())

('[{"type":"value_error","loc":["url_slug"],"msg":"Value error, Must not '
 'contain spaces","input":"my day trip","ctx":{"error":"Must not contain '
 'spaces"},"url":"https://errors.pydantic.dev/2.12/v/value_error"}]')


---

#### 4. Built-in Pydantic Metadata (Field constraints)


In [7]:
from typing import Annotated
from pydantic import BaseModel, Field, AfterValidator


def is_even(v: int) -> int:
    if v % 2 != 0:
        raise ValueError("Must be even")
    return v


# Combine:
# 1. Native Pydantic constraint (Greater than 0, Less than 100)
# 2. Custom Python logic (Must be even)
SpecialNumber = Annotated[int, Field(gt=0, lt=100), AfterValidator(is_even)]


class GameScore(BaseModel):
    score: SpecialNumber

In [8]:
# Valid
print(GameScore(score=88))

score=88


In [9]:
# Invalid (Odd number)
GameScore(score=89)  # -> Error: Must be even

ValidationError: 1 validation error for GameScore
score
  Value error, Must be even [type=value_error, input_value=89, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

In [10]:
# Invalid (Too high)
GameScore(score=150)  # -> Error: Input should be less than 100

ValidationError: 1 validation error for GameScore
score
  Input should be less than 100 [type=less_than, input_value=150, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/less_than